In [ ]:
# do all necessary imports for this chapter
import matplotlib.pyplot as plt
import numpy as np
from ncon import ncon
from scipy.optimize import minimize
from scipy.sparse.linalg import eigs, LinearOperator, gmres
from scipy.linalg import svd, polar, null_space
from functools import partial
from time import time
from tutorialFunctions import createMPS, normalizeMPS, fixedPoints, rightOrthonormalize, mixedCanonical, expVal2Uniform, expVal2Mixed

# [Tangent-space methods for uniform matrix product states](https://doi.org/10.21468/SciPostPhysLectNotes.7)

## 2. Finding ground states of local Hamiltonians

In the previous chapter, we stated that uniform MPS can be used to efficiently approximate low-energy states of one-dimensional systems with gapped local Hamiltonians. Having defined ways of representing and manipulating MPS, the logical next step is therefore to have a look at how exactly they can be used to find ground states. To this end, we consider a nearest-neighbour Hamiltonian $H$  of the form

$$H = \sum_n h_{n, n+1}$$

acting on an infinite one-dimensional system. Here, $h_{n,n+1}$ is a hermitian operator acting non-trivially on sites $n$ and $n+1$. As in any variational approach, the variational principle serves as a guide for finding ground-state approximations, dictating that the optimal MPS approximation of the ground state corresponds to the minimum of the expectation value of the energy,

$$ \min_A \frac{\left \langle \Psi(\bar{A}) \middle | H  \middle | \Psi(A) \right \rangle}{\left \langle \Psi(\bar{A}) \middle | \Psi(A)  \right \rangle}. $$

In the thermodynamic limit the energy diverges with system size, but, since we are working with translation-invariant states only, we should rather minimize the energy density. In the following we will always restrict our discussion to preoperly normalized states. Diagrammatically, the minimization problem can then be recast as

<center><img src="img/2minham.svg" alt="minimization of hamiltonian"></center>

In this notebook we illustratre numerical optimization strategies for minimizing this energy density directly.

### 2.1 The gradient

Any optimization problem relies on an efficient evaluation of the gradient, so the first thing to do is to compute this quantity. The objective function $f$ that we want to minimize is a real function of the complex-valued $A$, or equivalently, of the independent variables $A$ and $\bar{A}$. The gradient $g$ is then obtained by differentiating $f(\bar{A},A)$ with respect to $\bar{A}$,

$$
\begin{align}
g &= 2 \times \frac{\partial f(\bar{A},A) }{ \partial \bar{A} } \\
&= 2\times \frac{\partial_{\bar{A}} \left \langle \Psi(\bar{A}) \middle | h  \middle | \Psi(A) \right \rangle } {\left \langle \Psi(\bar{A}) \middle | \Psi(A)  \right \rangle} - 2\times \frac{\left \langle \Psi(\bar{A}) \middle | h  \middle | \Psi(A) \right \rangle} {\left \langle \Psi(\bar{A}) \middle | \Psi(A)  \right \rangle^2} \partial_{\bar{A}} \left \langle \Psi(\bar{A}) \middle | \Psi(A) \right \rangle ,\\
&= 2\times \frac{\partial_{\bar{A}}  \left \langle \Psi(\bar{A}) \middle | h  \middle | \Psi(A) \right \rangle - e \partial_{\bar{A}} \left \langle \Psi(\bar{A}) \middle | \Psi(A)  \right \rangle  } {\left \langle \Psi(\bar{A}) \middle | \Psi(A)  \right \rangle},\\
\end{align}
$$

where we have clearly indicated $A$ and $\bar{A}$ as independent variables and $e$ is the current energy density given by

$$
e = \frac{\left \langle \Psi(\bar{A}) \middle | h  \middle | \Psi(A) \right \rangle} {\left \langle \Psi(\bar{A}) \middle | \Psi(A)  \right \rangle}.
$$

If we make sure that the MPS is properly normalized and subtract the current energy density from every term in the hamiltonian, $h \leftarrow h - e$, the gradient takes on the simple form

$$ g = 2 \times \partial_{\bar{A}} \left \langle \Psi(\bar{A}) \middle | h  \middle | \Psi(A) \right \rangle.$$

Thus, the gradient is obtained by differentiating the expression

<center><img src="img/grad.svg" alt="gradient"></center>

with respect to $\bar{A}$. This gives rise to a sum over all sites, where in every term we differentiate with respect to one tensor $\bar{A}$ in the bra layer. Differentiating with respect to one $\bar{A}$ tensor amounts to leaving out that tensor, and interpreting the open legs as outgoing ones, i.e. each term looks like

<center><img src="img/gradTerm.svg" alt="gradient term"></center>

The full gradient is then obtained as an infinite sum over these terms. By dividing the terms into three different classes and doing some bookkeeping as illustrated below, we can eventually write this sum in a relatively simple closed form.

#### Terms of the 'center' kind
The first kind of terms that arise in the above expression for the gradient are obtained by differentiation with respect to an $\bar{A}$ tensor on the legs of the Hamiltonian operator. This results in two 'center' terms

<center><img src="img/centerTerms.svg" alt="center terms"></center>

In [ ]:
def gradCenterTerms(hTilde, A, l=None, r=None):
    """
    Calculate the value of the center terms.
    
        Parameters
        ----------
        hTilde : np.array (d, d, d, d)
            reduced Hamiltonian,
            ordered topLeft-topRight-bottomLeft-bottomRight.
        A : np.array (D, d, D)
            normalized MPS tensor with 3 legs,
            ordered left-bottom-right.
        l : np.array (D, D), optional
            left fixed point of transfermatrix,
            normalized.
        r : np.array (D, D), optional
            right fixed point of transfermatrix,
            normalized.
        
        Returns
        -------
        term1 : np.array (D, d, D)
            first term of gradient,
            ordered left-mid-right.
        term2 : np.array (D, d, D)
            second term of gradient,
            ordered left-mid-right.
    """
    
    return term1, term2

#### Terms of the 'left' kind
For the terms where we leave out an $\bar{A}$ tensor to the left of $h$, which we will call 'left' terms, we can contract everything to the left of this missing $\bar{A}$ with the left fixed point $l$, while everything to the right of $h$ can be contracted with right fixed point $r$.

In between these two outer parts of the network there remains a region where the regular MPS transfer matrix $E$ is applied a number of times. The action of this region is therefore captured by the operator $E^n$, where the power $n$ is determined by the seperation between the outer left and right parts for the specific term under consideration. When summing all left terms, the outer parts of the contraction always remain the same, while only the power $n$ differs for every term. Thus, summing all left terms corresponds to contracting the operator 

$$E_\text{sum} = 1 + E + E^2 + \dots = \frac{1}{1-E}$$

between the left and right outer parts. Here, we have naively used the geometric series to write the sum in a closed form. However, since by our normalization the transfer matrix has leading eigenvalue $1$, this resulting expression will diverge and is therefore ill-defined. We can get around this by introducing a regularized transfer matrix $\tilde{E}$ which is defined by subtracting the divergent part,

<center><img src="img/regTransfer.svg" alt="regularized transfer matrix"></center>

Since we have already shifted the energy density to have a zero expectation value, $h \leftarrow h - e$, it can easily be verified that the contribution of the leading divergent part vanishes in every left term, meaning that we can simply replace the original transfer matrix by its regularized version without changing any of the terms, and only then take the infinite sum which now has a well defined expression in terms of an inverse,

$$ E_\text{sum} \rightarrow \frac{1}{1-\tilde{E}} \equiv (1 - E)^P ,$$

where we have introduced the pseudo-inverse defined as $(1 - E)^P = (1-\tilde{E})^{-1}$.

Using this notation we can define the partial contraction

<center><img src="img/Rh.svg" alt="right effective environment"></center>

such that the sum of all left terms equals

<center><img src="img/leftTerms.svg" alt="left terms"></center>

If we would compute the partial contraction $R_h$ directly by explicitly computing the pseudo-inverse, this would entail a computational complexity $O(D^6)$. Instead, we can define $L_h$ as the solution of a linear system by multiplying both sides of the corresponding definition by $(1-\tilde{E})$. This results in an equation of the form $Ax = b$, which may be solved for $x$ by using Krylov-based iterative methods such as a Generalized Minimal RESidual (GMRES) algorithm. Note that these methods only require the action of $A = (1-\tilde{E})$ on a vector and not the full matrix $A$. This action can again be supplied to the linear solver using a function handle.

In [ ]:
def reducedHamUniform(h, A, l=None, r=None):
    """
    Regularize Hamiltonian such that its expectation value is 0.
    
        Parameters
        ----------
        h : np.array (d, d, d, d)
            Hamiltonian that needs to be reduced,
            ordered topLeft-topRight-bottomLeft-bottomRight.
        A : np.array (D, d, D)
            normalized MPS tensor with 3 legs,
            ordered left-bottom-right.
        l : np.array (D, D), optional
            left fixed point of transfermatrix,
            normalized.
        r : np.array (D, D), optional
            right fixed point of transfermatrix,
            normalized.
            
        Returns
        -------
        hTilde : np.array (d, d, d, d)
            reduced Hamiltonian,
            ordered topLeft-topRight-bottomLeft-bottomRight.
    """
    
    return hTilde

In [ ]:
def EtildeRight(A, l, r, v):
    """
    Implement the action of (1 - Etilde) on a right vector v.
    
        Parameters
        ----------
        A : np.array (D, d, D)
            normalized MPS tensor with 3 legs,
            ordered left-bottom-right.
        l : np.array (D, D), optional
            left fixed point of transfermatrix,
            normalized.
        r : np.array (D, D), optional
            right fixed point of transfermatrix,
            normalized.
        v : np.array (D**2)
            right matrix of size (D, D) on which
            (1 - Etilde) acts,
            given as a vector of size (D**2,)
        
        Returns
        -------
        vNew : np.array (D**2)
            result of action of (1 - Etilde)
            on a right matrix,
            given as a vector of size (D**2,)
    """
    
    # given as an example
    
    D = A.shape[0]
    
    # reshape to matrix
    v = v.reshape(D, D)
        
    # transfermatrix contribution
    transfer = ncon((A, np.conj(A), v), ([-1, 2, 1], [-2, 2, 3], [1, 3]))

    # fixed point contribution
    fixed = np.trace(l @ v) * r

    # sum these with the contribution of the identity
    vNew = v - transfer + fixed

    return vNew.reshape((D ** 2))

In [ ]:
def RhUniform(hTilde, A, l=None, r=None):
    """
    Find the partial contraction for Rh.
    
        Parameters
        ----------
        hTilde : np.array (d, d, d, d)
            reduced Hamiltonian,
            ordered topLeft-topRight-bottomLeft-bottomRight,
            renormalized.
        A : np.array (D, d, D)
            normalized MPS tensor with 3 legs,
            ordered left-bottom-right.
        l : np.array (D, D), optional
            left fixed point of transfermatrix,
            normalized.
        r : np.array (D, D), optional
            right fixed point of transfermatrix,
            normalized.
        
        Returns
        -------
        Rh : np.array (D, D)
            result of contraction,
            ordered top-bottom.
    """
    
    # given as an example
    
    D = A.shape[0]
    
    # if l, r not specified, find fixed points
    if l is None or r is None:
        l, r = fixedPoints(A)
    
    # construct b, which is the matrix to the right of (1 - E)^P in the figure above
    b = ncon((r, A, A, np.conj(A), np.conj(A), hTilde), ([4, 5], [-1, 2, 1], [1, 3, 4], [-2, 8, 7], [7, 6, 5], [2, 3, 8, 6]))
    
    # solve Ax = b for x
    A = LinearOperator((D ** 2, D ** 2), matvec=partial(EtildeRight, A, l, r))
    Rh = gmres(A, b.reshape(D ** 2))[0]
    
    return Rh.reshape((D, D))

In [ ]:
def gradLeftTerms(hTilde, A, l=None, r=None):
    """
    Calculate the value of the left terms.
    
        Parameters
        ----------
        hTilde : np.array (d, d, d, d)
            reduced Hamiltonian,
            ordered topLeft-topRight-bottomLeft-bottomRight,
            renormalized.
        A : np.array (D, d, D)
            MPS tensor with 3 legs,
            ordered left-bottom-right.
        l : np.array (D, D), optional
            left fixed point of transfermatrix,
            normalized.
        r : np.array (D, D), optional
            right fixed point of transfermatrix,
            normalized.
        
        Returns
        -------
        leftTerms : np.array (D, d, D)
            left terms of gradient,
            ordered left-mid-right.
    """
    
    return leftTerms

#### Terms of the 'right' kind

In a similar way, the terms where we leave out an $\bar{A}$ to the right of  $h$ can be evaluated by defining the partial contraction

<center><img src="img/Lh.svg" alt="Lh"></center>

which can again be found by solving a linear system, such that the sum of all right terms can be written as

<center><img src="img/rightTerms.svg" alt="rightTerms"></center>

In [ ]:
def EtildeLeft(A, l, r, v):
    """
    Implement the action of (1 - Etilde) on a left vector matrix v.
    
        Parameters
        ----------
        A : np.array (D, d, D)
            normalized MPS tensor with 3 legs,
            ordered left-bottom-right.
        l : np.array (D, D), optional
            left fixed point of transfermatrix,
            normalized.
        r : np.array (D, D), optional
            right fixed point of transfermatrix,
            normalized.
        v : np.array (D**2)
            right matrix of size (D, D) on which
            (1 - Etilde) acts,
            given as a vector of size (D**2,)
        
        Returns
        -------
        vNew : np.array (D**2)
            result of action of (1 - Etilde)
            on a left matrix,
            given as a vector of size (D**2,)
    """
    
    return vNew

In [ ]:
def LhUniform(hTilde, A, l=None, r=None):
    """
    Find the partial contraction for Lh.
    
        Parameters
        ----------
        hTilde : np.array (d, d, d, d)
            reduced Hamiltonian,
            ordered topLeft-topRight-bottomLeft-bottomRight,
            renormalized.
        A : np.array (D, d, D)
            MPS tensor with 3 legs,
            ordered left-bottom-right.
        l : np.array (D, D), optional
            left fixed point of transfermatrix,
            normalized.
        r : np.array (D, D), optional
            right fixed point of transfermatrix,
            normalized.
        
        Returns
        -------
        Lh : np.array (D, D)
            result of contraction,
            ordered bottom-top.
    """
    
    return Lh

In [ ]:
def gradRightTerms(hTilde, A, l=None, r=None):
    """
    Calculate the value of the right terms.
    
        Parameters
        ----------
        hTilde : np.array (d, d, d, d)
            reduced Hamiltonian,
            ordered topLeft-topRight-bottomLeft-bottomRight,
            renormalized.
        A : np.array (D, d, D)
            MPS tensor with 3 legs,
            ordered left-bottom-right.
        l : np.array (D, D), optional
            left fixed point of transfermatrix,
            normalized.
        r : np.array (D, D), optional
            right fixed point of transfermatrix,
            normalized.
        
        Returns
        -------
        rightTerms : np.array (D, d, D)
            right terms of gradient,
            ordered left-mid-right.
    """
    
    return rightTerms

#### The gradient

The full gradient is then found by summing the contributions of all three types of terms,

<center><img src="img/gradFull.svg" alt="gradient"></center>

In [ ]:
def gradient(h, A, l=None, r=None):
    """
    Calculate the gradient of the expectation value of h @ MPS A.
    
        Parameters
        ----------
        h : np.array (d, d, d, d)
            Hamiltonian,
            ordered topLeft-topRight-bottomLeft-bottomRight,
            renormalized.
        A : np.array (D, d, D)
            MPS tensor with 3 legs,
            ordered left-bottom-right.
        l : np.array (D, D), optional
            left fixed point of transfermatrix,
            normalized.
        r : np.array (D, D), optional
            right fixed point of transfermatrix,
            normalized.
        
        Returns
        -------
        grad : np.array (D, d, D)
            Gradient,
            ordered left-mid-right.
    """
    
    return grad

### 2.2 Gradient descent algorithms

The most straightforward way to use this expression for the gradient to find the ground state of a Hamiltonian is to implement a gradient-search method for minimizing the energy expecation value. The simplest such method is a steepest-descent search, where in every iteration the tensor $A$ is updated in the direction opposite to the gradient along a small step $\varepsilon$,

$$ A_{i+1} = A_i - \varepsilon g .$$

This procedure is repeated until we find the optimal MPS tensor $A^*$ for which the gradient vanishes. This approach can be improved upon by resorting to other optimization schemes such a conjugate-gradient or quasi-Newton methods. Below we demonstrate both a simple steepest-descent with a fixed step size, as well as an approach using routines supplied by the scipy package in `scipy.optimize`.

In [ ]:
def groundStateGradDescent(h, D, eps=1e-1, A0=None, tol=1e-4, maxIter=1e4, verbose=True):
    """
    Find the ground state using gradient descent.
    
        Parameters
        ----------
        h : np.array (d, d, d, d)
            Hamiltonian to minimize,
            ordered topLeft-topRight-bottomLeft-bottomRight.
        D : int
            Bond dimension
        eps : float
            Stepsize.
        A0 : np.array (D, d, D)
            normalized MPS tensor with 3 legs,
            ordered left-bottom-right,
            initial guess.
        tol : float
            Tolerance for convergence criterium.
        
        Returns
        -------
        E : float
            expectation value @ minimum
        A : np.array (D, d, D)
            ground state MPS,
            ordered left-mid-right.
    """
    
    d = h.shape[0]
    
    # if no initial value, choose random
    if A0 is None:
        A0 = createMPS(D, d)
        A0 = normalizeMPS(A0)
    
    # calculate gradient
    g = gradient(h, A0)
    g0 = np.zeros((D, d, D))
    
    A = A0
    
    i = 0
    while not(np.linalg.norm(g) < tol):
        # do a step
        A = A - eps * g
        A = normalizeMPS(A)
        i += 1
        
        if verbose and not(i % 100):
            E = np.real(expVal2Uniform(h, A))
            print('iteration:\t{:d}\tenergy:\t{:.12f}\tgradient norm:\t{:.4e}'.format(i, E, np.linalg.norm(g)))
        
        # calculate new gradient
        g = gradient(h, A)
        
        if i > maxIter:
            print('Warning: gradient descent did not converge!')
            break
    
    # calculate ground state energy
    E = np.real(expVal2Uniform(h, A))
    
    return E, A

In order to use the minimize function from scipy for this purpose, one must keep in mind the fact that minimize requires an objective function that maps a real vector to a scalar. In particular, complex tensors must be given as reals vectors in the input, and must be again represented as real vectors in the output.

In [ ]:
def groundStateMinimize(h, D, A0=None, tol=1e-4):
    """
    Find the ground state using a scipy minimizer.
    
        Parameters
        ----------
        h : np.array (d, d, d, d)
            Hamiltonian to minimize,
            ordered topLeft-topRight-bottomLeft-bottomRight.
        D : int
            Bond dimension
        A0 : np.array (D, d, D)
            MPS tensor with 3 legs,
            ordered left-bottom-right,
            initial guess.
        tol : float
            Relative convergence criterium.
        
        Returns
        -------
        E : float
            expectation value @ minimum
        A : np.array (D, d, D)
            ground state MPS,
            ordered left-mid-right.
    """
    
    d = h.shape[0]
    
    def unwrapper(varA):
        """
        Unwraps real MPS vector to complex MPS tensor.
        
            Parameters
            ----------
            varA : np.array (2 * D * d * D)
                MPS tensor in real vector form.
            D : int
                Bond dimension.
            d : int
                Physical dimension.
                
            Returns
            -------
            A : np.array (D, d, D)
                MPS tensor with 3 legs,
                ordered left-bottom-right.
        """
        
        # unpack real and imaginary part
        Areal = varA[:D ** 2 * d]
        Aimag = varA[D ** 2 * d:]
        
        A = Areal + 1.0j * Aimag
        
        return np.reshape(A, (D, d, D))
    
    def wrapper(A):
        """
        Wraps MPS tensor to real MPS vector.
        
            Parameters
            ----------
            A : np.array (D, d, D)
                MPS tensor,
                ordered left-bottom-right
            
            Returns
            -------
            varA : np.array (2 * D * d * D)
                MPS tensor in real vector form.
        """
        
        # split into real and imaginary part
        Areal = np.real(A)
        Aimag = np.imag(A)
        
        # combine into vector
        varA = np.concatenate( (Areal.reshape(-1), Aimag.reshape(-1)) )
        
        return varA
    
    # if no initial MPS, take random one
    if A0 is None:
        A0 = createMPS(D, d)
        A0 = normalizeMPS(A0)
    
    # define f for minimize in scipy
    def f(varA):
        """
        Function to optimize via minimize.
        
            Parameters
            ----------
            varA : np.array (2 * D * d * D)
                MPS tensor in real vector form.
            
            Returns
            -------
            e : float
                function value @varA
            g : np.array (2 * D * d * D)
                gradient vector @varA
        """
        
        # unwrap varA
        A = unwrapper(varA)
        A = normalizeMPS(A)
        
        # calculate fixed points
        l, r = fixedPoints(A)
        
        # calculate function value and gradient
        e = np.real(expVal2Uniform(h, A, l, r))
        g = gradient(h, A, l, r)
        
        # wrap g
        g = wrapper(g)
        
        return e, g
    
    # calculate minimum
    result = minimize(f, wrapper(A0), jac=True, tol=tol)
    
    # unpack result
    E = result.fun
    A = unwrapper(result.x)
    
    return E, A

To demonstrate these methods, we now have a look the specific case of the antiferromagnetic spin-1 Heisenberg model in one dimension. To this end we first define the spin-1 Heisenberg Hamiltonian:

In [ ]:
def Heisenberg(Jx, Jy, Jz, hz):
    """
    Construct the spin-1 Heisenberg Hamiltonian for given couplings.
    
        Parameters
        ----------
        Jx : float
            Coupling strength in x direction
        Jy : float
            Coupling strength in y direction
        Jy : float
            Coupling strength in z direction
        hz : float
            Coupling for Sz terms

        Returns
        -------
        h : np.array (3, 3, 3, 3)
            Spin-1 Heisenberg Hamiltonian.
    """
    Sx = np.array([[0, 1, 0], [1, 0, 1], [0, 1, 0]]) / np.sqrt(2)
    Sy = np.array([[0, -1, 0], [1, 0, -1], [0, 1, 0]]) * 1.0j /np.sqrt(2)
    Sz = np.array([[1, 0, 0], [0, 0, 0], [0, 0, -1]])
    I = np.eye(3)

    return -Jx*ncon((Sx, Sx), ([-1, -3], [-2, -4]))-Jy*ncon((Sy, Sy), ([-1, -3], [-2, -4]))-Jz*ncon((Sz, Sz), ([-1, -3], [-2, -4])) \
            - hz*ncon((I, Sz), ([-1, -3], [-2, -4])) - hz*ncon((Sz, I), ([-1, -3], [-2, -4]))

In [ ]:
d, D = 3, 12
A = createMPS(D, d)
A = normalizeMPS(A)

h = Heisenberg(-1, -1, -1, 0)

# energy optimization using naive gradient descent
# for D=12 or higher: tolerance lower than 1e-2 gives very long runtimes
print('Gradient descent optimization:\n')
t0 = time()
E1, A1 = groundStateGradDescent(h, D, eps=1e-1, A0=A, tol=1e-2, maxIter=1e4)
print('Time until convergence:', time()-t0, 's')
print('Computed energy:', E1, '\n')

# energy optimization using scipy optimizer
# for D=12 and tolerance 1e-5: runtime of somewhere around 100s
print('Optimization using scipy minimize:\n')
t0 = time()
E2, A2 = groundStateMinimize(h, D, A0=A, tol=1e-4)
print('Time until convergence:', time()-t0, 's')
print('Computed energy:', E2, '\n')

### 2.3 The VUMPS algorithm

In the previous section we have derived an expression for the gradient starting from an MPS in the uniform gauge, which corresponds to an object that lives in the space of MPS tensors. We now discuss how to improve upon direct optimization schemes based on this form of the gradient by exploiting the structure of the MPS manifold as well as the mixed gauge for MPS.

Indeed, while the gradient in the above form indicates a direction in the space of complex tensors in which the energy decreases, intuitively it would make more sense if we could find a way to interpret the gradient as a direction *along the MPS manifold* along which we can decrease the energy. This can be achieved by interpreting the gradient as a *tangent vector in the tangent space to the MPS manifold*. By formulating the energy optimization in terms of this tangent space gradient written in mixed gauge, one arives at the [VUMPS](https://doi.org/10.1103/PhysRevB.97.045145) algorithm (which stand for 'variational uniform matrix product states'). The precise derivation of the tangent space gradient in mixed gauge falls beyond the scope of this tutorial, and can be found in the [lecture notes](https://doi.org/10.21468/SciPostPhysLectNotes.7). Instead we will simply illustrate the implementation of the VUMPS algorithm given the mixed gauge tangent space gradient.

Most of the following required steps will be reminiscent of those outlined above, where we now consistently work in the mixed gauge. We start off by implementing the regularization of the two-site Hamiltonian in the mixed gauge.

In [ ]:
def reducedHamMixed(h, Ac, Ar):
    """
    Regularize Hamiltonian such that its expectation value is 0.
    
        Parameters
        ----------
        h : np.array (d, d, d, d)
            Hamiltonian that needs to be reduced,
            ordered topLeft-topRight-bottomLeft-bottomRight.
        Ac : np.array (D, d, D)
            MPS tensor with 3 legs,
            ordered left-bottom-right,
            center gauged.
        Ar : np.array (D, d, D)
            MPS tensor with 3 legs,
            ordered left-bottom-right,
            right gauged.

        Returns
        -------
        hTilde : np.array (d, d, d, d)
            reduced Hamiltonian,
            ordered topLeft-topRight-bottomLeft-bottomRight.
    """
    
    return hTilde

The variational optimum of the energy is characterized by the condition that the gradient is zero at this point. Writing the tangent space gradient as $G$, we now wish to formulate an algorithm which minimizes the error measure

$$ \varepsilon = \left( \boldsymbol{G}^\dagger \boldsymbol{G} \right)^{1/2} $$

in an efficient way. The explicit form of the tangent space gradient in mixed gauge is given by

$$ G = A^\prime_{C} - A_L C^\prime = A^\prime_{C} - C^\prime A_R, $$

where $A^\prime_{C}$ and $C^\prime$ are defined as

<center><img src="img/Acprime.svg" alt="Ac prime"></center>

and

<center><img src="img/Cprime.svg" alt="C prime"></center>

Here, we again use $L_h$ and $R_h$ to indicate the partial contractions

<center><img src="img/LhMixed.svg" alt="Lh mixed gauge"></center>

and

<center><img src="img/RhMixed.svg" alt="Rh mixed gauge"></center>

where the transfer matrices $E^L_L$ and $E^R_R$ appearing in these expressions now contain only left-gauged and right-gauged MPS tensors $A_L$ and $A_R$ respectively.

If we interpret the two terms appearing in the tangent space gradient as defining the effective Hamiltonians $H_{A_C}(\cdot)$ and $H_C(\cdot)$ such that

$$
\begin{align}
H_{A_C}(A_C) = A_C^\prime \\
H_C(C) = C^\prime ,
\end{align}
$$

we can characterize the variational optimum in terms of the fixed points of these operators. Indeed, since the tangent space gradient should be zero at the variational optimum, this point satisfies $A_C' = A_L C' = C' A_R$. This implies that the optimal MPS should obey the following set of equations,

$$
\begin{align}
H_{A_C}(A_C) \propto A_C \\
H_C(C) \propto C \\
A_C = A_L C = C A_R ,
\end{align}
$$

meaning that the optimal MPS should correspond to a fixed point of the effective Hamiltonians $H_{A_C}$ and $H_C$ and satisfy the mixed gauge condition. The VUMPS algorithm then consists of an iterative method for finding a set $\{A_L, A_C, A_R, C\}$ that satisfies these equations simultaneously.

#### Defining the required operators

Similar to before, we again have to compute the contributions of the left and right environment terms $L_h$ and $R_h$ given above. We therefore require function handles defining the action of the left (resp. right) transfer matrix $E^L_L$ (resp. $E^R_R$) on a left (resp. right) matrix. To this end, we can simply reuse the implementations `EtildeLeft` and `EtildeRight` defined above, if we take into account that the left (resp. right) fixed point of $E^L_L$ (resp. $E^R_R$) is the identity while its right (resp. left) fixed point is precisely $C C^\dagger$ (resp. $C^\dagger C$). This last fact follows immediately from the mixed gauge condition.

In [ ]:
def LhMixed(hTilde, Al, C, tol=1e-5):
    """
    Calculate Lh, for a given MPS in mixed gauge.
    
        Parameters
        ----------
        hTilde : np.array (d, d, d, d)
            reduced Hamiltonian,
            ordered topLeft-topRight-bottomLeft-bottomRight,
            renormalized.
        Al : np.array (D, d, D)
            MPS tensor with 3 legs,
            ordered left-bottom-right,
            left-orthonormal.
        C : np.array (D, D)
            Center gauge with 2 legs,
            ordered left-right.
        tol : float, optional
            tolerance for gmres
            
        Returns
        -------
        Lh : np.array (D, D)
            result of contraction,
            ordered bottom-top.
    
    """
    
    return Lh

In [ ]:
def RhMixed(hTilde, Ar, C, tol=1e-5):
    """
    Calculate Rh, for a given MPS in mixed gauge.
    
        Parameters
        ----------
        hTilde : np.array (d, d, d, d)
            reduced Hamiltonian,
            ordered topLeft-topRight-bottomLeft-bottomRight,
            renormalized.
        Ar : np.array (D, d, D)
            MPS tensor with 3 legs,
            ordered left-bottom-right,
            right-orthonormal.
        C : np.array (D, D)
            Center gauge with 2 legs,
            ordered left-right.
        tol : float, optional
            tolerance for gmres
            
        Returns
        -------
        Rh : np.array (D, D)
            result of contraction,
            ordered top-bottom.
    """
    
    return Rh

Next we implement the actions of the effective Hamiltonians $H_{A_C}$ and $H_{C}$ defined above,

<center><img src="img/H_Ac.svg" alt="H_Ac"></center>

<center><img src="img/H_C.svg" alt="H_C"></center>

In [ ]:
def H_Ac(hTilde, Al, Ar, Lh, Rh, v):
    """
    Action of the effective Hamiltonian for Ac (131) on a vector.

        Parameters
        ----------
        hTilde : np.array (d, d, d, d)
            reduced Hamiltonian,
            ordered topLeft-topRight-bottomLeft-bottomRight,
            renormalized.
        Al : np.array (D, d, D)
            MPS tensor with 3 legs,
            ordered left-bottom-right,
            left-orthonormal.
        Ar : np.array (D, d, D)
            MPS tensor with 3 legs,
            ordered left-bottom-right,
            right-orthonormal.
        Lh : np.array (D, D)
            left environment,
            ordered bottom-top.
        Rh : np.array (D, D)
            right environment,
            ordered top-bottom.
        v : np.array (D, d, D)
            Tensor of size (D, d, D)

        Returns
        -------
        H_AcV : np.array (D, d, D)
            Result of the action of H_Ac on the vector v,
            representing a tensor of size (D, d, D)

    """

    # given as an example
    
    # first term
    term1 = ncon((Al, v, np.conj(Al), hTilde), ([4, 2, 1], [1, 3, -3], [4, 5, -1], [2, 3, 5, -2]))

    # second term
    term2 = ncon((v, Ar, np.conj(Ar), hTilde), ([-1, 2, 1], [1, 3, 4], [-3, 5, 4], [2, 3, -2, 5]))

    # third term
    term3 = ncon((Lh, v), ([-1, 1], [1, -2, -3]))

    # fourth term
    term4 = ncon((v, Rh), ([-1, -2, 1], [1, -3]))

    # sum
    H_AcV = term1 + term2 + term3 + term4

    return H_AcV

In [ ]:
def H_C(hTilde, Al, Ar, Lh, Rh, v):
    """
    Action of the effective Hamiltonian for Ac (131) on a vector.

        Parameters
        ----------
        hTilde : np.array (d, d, d, d)
            reduced Hamiltonian,
            ordered topLeft-topRight-bottomLeft-bottomRight,
            renormalized.
        Al : np.array (D, d, D)
            MPS tensor with 3 legs,
            ordered left-bottom-right,
            left-orthonormal.
        Ar : np.array (D, d, D)
            MPS tensor with 3 legs,
            ordered left-bottom-right,
            right-orthonormal.
        Lh : np.array (D, D)
            left environment,
            ordered bottom-top.
        Rh : np.array (D, D)
            right environment,
            ordered top-bottom.
        v : np.array (D, D)
            Matrix of size (D, D)

        Returns
        -------
        H_CV : np.array (D, D)
            Result of the action of H_C on the matrix v.

    """

    return H_CV

#### Implementing the VUMPS algorithm

In order to find a set $\{A_L^*, A_C^*, A_R^*, C^*\}$ that satisfies the VUMPS fixed point equations given above, we use an iterative method in which each iteration consists of the following steps, each time starting from a given set $\{A_L, A_C, A_R, C\}$:

1. Solve the eigenvalue equations for $H_{A_C}$ and $H_C$, giving new center tensors $\tilde{A}_C$ and $\tilde{C}$.

2. From these new center tensors, construct a set $\{\tilde{A}_L, \tilde{A}_R, \tilde{A}_C, \tilde{C}\}$.

3. Update the set of tensors $\{A_L, A_C, A_R, C\} \leftarrow \{\tilde{A}_L, \tilde{A}_C, \tilde{A}_R, \tilde{C}\}$ and evaluate the norm of the gradient $\varepsilon = \left | \left | H_{A_C} (A_C) - A_L H_C(C) \right | \right |$.

4. If the norm of the gradient lies above the given tolerance, repeat.

##### Updating the center tensors

We start by defining a routine `calcNewCenter` which finds the new center tensors $\tilde{A}_C$ and $\tilde{C}$ by solving the eigenvalue problem defined by the effective Hamiltonians implemented above.

In [ ]:
def calcNewCenter(hTilde, Al, Ac, Ar, C, Lh=None, Rh=None, tol=1e-5):
    """
    Find new guess for Ac and C as fixed points of the maps H_Ac and H_C.
    
        Parameters
        ----------
        hTilde : np.array (d, d, d, d)
            reduced Hamiltonian,
            ordered topLeft-topRight-bottomLeft-bottomRight,
            renormalized.
        Al : np.array (D, d, D)
            MPS tensor zith 3 legs,
            ordered left-bottom-right,
            left orthonormal.
        Ar : np.array (D, d, D)
            MPS tensor zith 3 legs,
            ordered left-bottom-right,
            right orthonormal.
        Ac : np.array (D, d, D)
            MPS tensor zith 3 legs,
            ordered left-bottom-right,
            center gauge.
        C : np.array (D, D)
            Center gauge with 2 legs,
            ordered left-right.
        Lh : np.array (D, D)
            left environment,
            ordered bottom-top.
        Rh : np.array (D, D)
            right environment,
            ordered top-bottom.
        tol : float, optional
            current tolerance
            
        Returns
        -------
        AcTilde : np.array (D, d, D)
            MPS tensor zith 3 legs,
            ordered left-bottom-right,
            center gauge.
        CTilde : np.array (D, D)
            Center gauge with 2 legs,
            ordered left-right.
    """
    
    return AcTilde, CTilde

##### Extract a new set of mixed-gauge MPS tensors

Once we have new center tensors, we can use these to construct a new set of mixed-gauge MPS tensors. To do this in a stable way, we will determine the global updates $\tilde{A}_L$ and $\tilde{A}_R$ as the left and right isometric tensors that minimize

$$
\begin{align}
\varepsilon_L = \min ||\tilde{A}_C - \tilde{A}_L \tilde{C}||_2 \\
\varepsilon_R = \min ||\tilde{A}_C - \tilde{C} \tilde{A}_L||_2 .
\end{align}
$$

This can be achieved in a robust and close to optimal way by making use of the left and right polar decompositions

$$
\begin{align}
\tilde{A}_C = U^l_{A_C} P^l_{A_C}, \qquad \tilde{C} = U^l_{C} P^l_{C}, \\
\tilde{A}_C = P^r_{A_C}  U^r_{A_C} , \qquad \tilde{C} = P^r_{C} U^r_{C},
\end{align}
$$

to obtain

$$ \tilde{A}_L = U^l_{A_C} (U^l_C)^\dagger, \qquad \tilde{A}_R = (U^r_C)^\dagger U^r_{A_C}. $$

In order to give the  procedure some additional stability, we may also choose to use the $\tilde{A}_L$ obtained with these polar decompositions to compute the tensors $\tilde{A}_R$ and $\tilde{A}_C$ by right orthonormalization of this $\tilde{A}_L$. This approach ensures that the MPS satisfies the mixed gauge condition at all times, improving the overal stabilitiy of the VUMPS algorithm. This procedure is implemented in the `minAcC` routine.

In [ ]:
def minAcC(AcTilde, CTilde, tol=1e-5):
    """
    Find Al and Ar corresponding to Ac and C, according to algorithm 5 in the lecture notes.
    
        Parameters
        ----------
        AcTilde : np.array (D, d, D)
            MPS tensor with 3 legs,
            ordered left-bottom-right,
            new guess for center gauge. 
        CTilde : np.array (D, D)
            Center gauge with 2 legs,
            ordered left-right,
            new guess for center gauge
        
        Returns
        -------
        Al : np.array (D, d, D)
            MPS tensor zith 3 legs,
            ordered left-bottom-right,
            left orthonormal.
        Ar : np.array (D, d, D)
            MPS tensor zith 3 legs,
            ordered left-bottom-right,
            right orthonormal.
        Ac : np.array (D, d, D)
            MPS tensor with 3 legs,
            ordered left-bottom-right,
            center gauge. 
        C : np.array (D, D)
            Center gauge with 2 legs,
            ordered left-right,
            center gauge
    
    """
    
    return Al, Ac, Ar, C

##### Evaluating the norm of the gradient

As a last step, we use the routine `gradientNorm` to compute the norm of the tangent space gradient in order to check if the procedure has converged.

In [ ]:
def gradientNorm(hTilde, Al, Ac, Ar, C, Lh, Rh):
    """
    Calculate the norm of the gradient.
    
        Parameters
        ----------
        hTilde : np.array (d, d, d, d)
            reduced Hamiltonian,
            ordered topLeft-topRight-bottomLeft-bottomRight,
            renormalized.
        Al : np.array (D, d, D)
            MPS tensor zith 3 legs,
            ordered left-bottom-right,
            left orthonormal.
        Ar : np.array (D, d, D)
            MPS tensor zith 3 legs,
            ordered left-bottom-right,
            right orthonormal.
        Ac : np.array (D, d, D)
            MPS tensor zith 3 legs,
            ordered left-bottom-right,
            center gauge.
        C : np.array (D, D)
            Center gauge with 2 legs,
            ordered left-right.
        Lh : np.array (D, D)
            left environment,
            ordered bottom-top.
        Rh : np.array (D, D)
            right environment,
            ordered top-bottom.
        
        Returns
        -------
        norm : float
            norm of the gradient @Al, Ac, Ar, C
    """
    
    # given
    
    D = Al.shape[0]
    d = Al.shape[1]
    
    # calculate update on Ac and C using maps H_Ac and H_c
    AcUpdate = H_Ac(hTilde, Al, Ar, Lh, Rh, Ac)
    CUpdate = H_C(hTilde, Al, Ar, Lh, Rh, C)
    AlCupdate = ncon((Al, CUpdate), ([-1, -2, 1], [1, -3]))
    
    norm = np.linalg.norm(AcUpdate - AlCupdate)
    
    return norm

Finally, this allows to implement the VUMPS algorithm.

In [ ]:
def vumps(h, D, A0=None, tol=1e-4, tolFactor=1e-1, verbose=True):
    """
    Find the ground state of a given Hamiltonian using VUMPS.
    
        Parameters
        ----------
        h : np.array (d, d, d, d)
            Hamiltonian to minimize,
            ordered topLeft-topRight-bottomLeft-bottomRight.
        D : int
            Bond dimension
        A0 : np.array (D, d, D)
            MPS tensor with 3 legs,
            ordered left-bottom-right,
            initial guess.
        tol : float
            Relative convergence criterium.
        
        Returns
        -------
        E : float
            expectation value @ minimum
        Al : np.array (D, d, D)
            MPS tensor with 3 legs,
            ordered left-bottom-right,
            left orthonormal.
        Ar : np.array (D, d, D)
            MPS tensor with 3 legs,
            ordered left-bottom-right,
            right orthonormal.
        Ac : np.array (D, d, D)
            MPS tensor with 3 legs,
            ordered left-bottom-right,
            center gauge.
        C : np.array (D, D)
            Center gauge with 2 legs,
            ordered left-right.
    """
    
    return E, Al, Ac, Ar, C

We can again test this implementation on the spin-1 Heisenberg antiferromagnet.

In [ ]:
d, D = 3, 12
A = createMPS(D, d)
A = normalizeMPS(A)

h = Heisenberg(-1, -1, -1, 0)

# energy optimization using VUMPS
print('Energy optimization using VUMPS:\n')
t0 = time()
E, Al, Ac, Ar, C = vumps(h, D, A0=A, tol=1e-4, tolFactor=1e-2, verbose=True)
print('\nTime until convergence:', time()-t0, 's\n')
print('Computed energy:', E, '\n')

Having obtained this ground state MPS, it is worthwile to have a look at the corresponding entanglement spectrum.

In [ ]:
_, S, _ = svd(C) # singular values of center matrix give entanglement spectrum
plt.figure(dpi=120)
plt.title('Entanglement spectrum of ground state')
plt.scatter(np.arange(D)+1, S, marker='x')
plt.yscale('log')
plt.show()

We can clearly see that the entanglement spectrum consists of degenerate groups, which reflects an underlying symmetry in the ground state of the spin-1 Heisenberg antiferromagnet.

### 2.4 Elementary excitations

#### Quasiparticle ansatz

The methods described above can be extended beyond computing the ground state. We briefly discuss how one can also study excitations on top of a given ground state. For this, we introduce the MPS quasiparticle ansatz, given by

<center><img src="img/excitation.svg" alt="quasiparticle ansatz"></center>

This ansatz cosists of defining a new state by changing one $A$ tensor of the ground state at site $n$ and taking a momentum superposition.

Before describing how to optimize the tensor $B$, it is worthwile to investigate the corresponding variational space in a bit more detail. First, we note that this excitation ansatz can be interpreted as nothing more than a boosted version of a tangent vector to the MPS manifold. In particular, this means that we will be able to apply all kinds of useful tricks and manipulations to the tensor $B$ (cfr. the [lecture notes](https://doi.org/10.21468/SciPostPhysLectNotes.7) for an introduction to tangent vectors and their properties). For example, we can see that $B$ has gauge degrees of freedom, as the corresponding excited state is invariant under an additive gauge transformation of the form

<center><img src="img/gaugeExcitation.svg" alt="gauge transform excitation"></center>

where $Y$ is an arbitrary $D \times D$ matrix. This gauge freedom can be eliminated, thereby removing the zero modes in the variational subspace, by imposing a *left gauge-fixing condition*

<center><img src="img/gaugeFix.svg" alt="gauge fix"></center>

If we parametrize the tensor $B$ as

<center><img src="img/VlX.svg" alt="VlX"></center>

where $V_L$ is the $ D \times d \times D(d-1)$ tensor corresponding to the $D(d-1)$-dimensional null space of $A_L$ satisfying

<center><img src="img/Vl.svg" alt="Vl"></center>

then the gauge condition is automatically satisfied. In particular, this fixing of the gauge freedom ensures that the excitation is orthogonal to the ground state,

<center><img src="img/excitationOrth.svg" alt="excitationOrth"></center>

In this form, we have put forward an ansatz for an excited state characterized by a single $D(d-1) \times D$ matrix $X$ such that

1. All gauge degrees of freedom are fixed.
2. All zero modes in the variational subspace are removed.
3. Calculating the norm becomes straightforward.
4. The excitation is orthogonal to the ground state.

#### Solving the eigenvalue problem

Having introduced an excitation  ansatz which has all the right properties and is defined in terms of a single matrix $X$, all that is left to do is to minimize the energy function,

$$  \min_{X} \frac{\left \langle \Phi_p(X) \middle | H  \middle | \Phi_p(X) \right \rangle}{\left \langle \Phi_p(X) \middle | \Phi_p(X)  \right \rangle}. $$

As both the numerator and the denominator are quadratic functions of the variational parameters $X$, this optimization problem reduces to solving a generalized eigenvalue problem

$$ H_{\text{eff}}(q) X = \omega N_{\text{eff}}(q) X, $$

where the effective energy and normalization matrices are defined as

$$
\begin{align}
& 2\pi\delta(p-p') (\boldsymbol{X'})^\dagger H_{\text{eff}}(q) \boldsymbol{X} = \left \langle \Phi_{p'}(X') \middle | H  \middle | \Phi_p(X) \right \rangle \\
& 2\pi\delta(p-p') (\boldsymbol{X'})^\dagger N_{\text{eff}}(q) \boldsymbol{X} = \left \langle \Phi_{p'}(X') \middle | \Phi_p(X) \right \rangle,
\end{align}
$$

and $\boldsymbol{X}$ denotes a vectorized version of the matrix $X$. Since the overlap between two excited states is of the simple Euclidean form (cfr. the [lecture notes](https://doi.org/10.21468/SciPostPhysLectNotes.7)), the effective normalization matrix reduces to the unit matrix, and we are left with an ordinary eigenvalue problem.

To solve this eigenvalue problem, we need to find an expression for $H_{\text{eff}}$, or rather of the action thereof on a trial vector $\boldsymbol{Y}$. In order to find this action we first transform the vector $\boldsymbol{X}$ into a tensor $B$ by contracting its corresponding matrix with the right leg of $V_L$, and then compute all different contributions that pop up in a matrix element of the form $\left \langle \Phi_p(B') \middle | H  \middle | \Phi_p(B) \right \rangle$. This procedure is similar to what we have done when computing the gradient above, where we now need to take into account all different positions of the nearest-neighbor operator $h$ of the Hamiltonian, the input tensor $B$ and the output. Though slightly more involved than before, we can again define the following partion contractions

<center><img src="img/LhMixed.svg" alt="LhMixed"></center>

<br>

<center><img src="img/RhMixed.svg" alt="RhMixed"></center>

<br>

<center><img src="img/LB.svg" alt="LB"></center>

<br>

<center><img src="img/RB.svg" alt="RB"></center>

<br>

<center><img src="img/L1.svg" alt="L1"></center>

<br>

<center><img src="img/R1.svg" alt="R1"></center>

Using these partial contractions, we find the action of the effective energy matrix on a given input tensor $B(Y)$ as

<center><img src="img/HeffExcitation.svg" alt="HeffExcitation"></center>

In the last step, we need the action of $H_{\text{eff}}(p)$ on the vector $\boldsymbol{Y}$, so we need to perform a last contraction

<center><img src="img/quasi_inveff.svg" alt="quasi_inveff"></center>

The total procedure is implemented in the routine `quasiParticle`.

In [ ]:
def quasiParticle(h, Al, Ar, Ac, C, p, num):

    tol, D, d = 1e-12, Al.shape[0], Al.shape[1]
    # renormalize hamiltonian and find left and right environments
    hTilde = reducedHamMixed(h, Ac, Ar)
    Lh = LhMixed(hTilde, Al, C, tol)
    Rh = RhMixed(hTilde, Ar, C, tol)
    
    def ApplyHeff(x):
        
        x = np.reshape(x, (D*(d-1), D))
        B = ncon((Vl, x), ([-1, -2, 1], [1, -3]))
        
        def ApplyELR(x, p):
            x = x.reshape((D,D))
            overlap = ncon((np.conj(C), x),([1, 2], [1, 2]))
            y = ncon((Al, np.conj(Ar), x), ([-1, 3, 1], [-2, 3, 2], [1, 2]))
            y = x - np.exp(1j*p) * (y - overlap * C)
            y = y.reshape(-1)
            return y

        def ApplyERL(x, p):
            x = x.reshape((D,D))
            overlap=ncon((np.conj(C), x), ([1, 2], [1, 2]))
            y = ncon((x, Ar, np.conj(Al)), ([1, 2], [2, 3, -2], [1, 3, -1]))
            y = x - np.exp(1j*p) * (y - overlap * C)
            y = y.reshape(-1)
            return y

        
        # right disconnected
        right = ncon((B, np.conj(Ar)), ([-1, 2, 1], [-2, 2, 1]))
        handleApplyELR = LinearOperator((D**2, D**2), matvec=lambda v: ApplyELR(v,p))
        right = gmres(handleApplyELR, right.reshape(-1), tol=tol)[0]
        right = right.reshape((D,D))
        
        # left disconnected
        left = \
            1*ncon((Lh, B, np.conj(Al)), ([1,2], [2,3,-2],[1,3,-1]))+\
            1*ncon((Al, B, np.conj(Al), np.conj(Al), hTilde), ([1,2,4],[4,5,-2],[1,3,6],[6,7,-1],[3,7,2,5]))+\
            np.exp(-1j*p)*ncon((B, Ar, np.conj(Al), np.conj(Al), hTilde), ([1,2,4],[4,5,-2],[1,3,6],[6,7,-1],[3,7,2,5]))
        handleApplyERL = LinearOperator((D**2, D**2), matvec=lambda v: ApplyERL(v, -p))
        left = gmres(handleApplyERL, left.reshape(-1), tol=tol)[0]
        left = left.reshape((D,D))
        
        y = \
            1*ncon((B,Ar,np.conj(Ar),hTilde),([-1,2,1],[1,3,4],[-3,5,4],[-2,5,2,3]))+\
            np.exp(1j*p)*ncon((Al,B,np.conj(Ar),hTilde),([-1,2,1],[1,3,4],[-3,5,4],[-2,5,2,3]))+\
            np.exp(-1j*p)*ncon((B,Ar,np.conj(Al),hTilde),([4,3,1],[1,2,-3],[4,5,-1],[5,-2,3,2]))+\
            1*ncon((Al,B,np.conj(Al),hTilde),([4,3,1],[1,2,-3],[4,5,-1],[5,-2,3,2]))+\
            np.exp(1j*p)*ncon((Al,Al,np.conj(Al),right,hTilde),([1,2,4],[4,5,6],[1,3,-1],[6,-3],[3,-2,2,5]))+\
            np.exp(2*1j*p)*ncon((Al,Al,np.conj(Ar),right,hTilde),([-1,6,5],[5,3,2],[-3,4,1],[2,1],[-2,4,6,3]))+\
            1*ncon((Lh,B),([-1,1],[1,-2,-3]))+\
            1*ncon((B,Rh),([-1,-2,1],[1,-3]))+\
            np.exp(-1j*p)*ncon((left,Ar),([-1,1],[1,-2,-3]))+\
            np.exp(+1j*p)*ncon((Lh,Al,right),([-1,1],[1,-2,2],[2,-3]))
            
        y = ncon((y, np.conj(Vl)), ([1, 2, -2], [1, 2, -1]))
        y = y.reshape(-1)
        
        return y
    
    # find reduced parametrization
    L = np.reshape(np.moveaxis(np.conj(Al), -1, 0), (D, D*d))
    Vl = np.reshape(null_space(L), (D, d, D*(d-1)))
    handleHeff = LinearOperator((D**2*(d-1), D**2*(d-1)), matvec=lambda x: ApplyHeff(x))
    e, x = eigs(handleHeff, k=num, which='SR')
    
    return x, e


We can use this to compute the Haldane gap on top of the ground state of the spin-1 Heisenberg antiferromagnet we have just obtained using VUMPS.

In [ ]:
p = np.pi
num = 3;
x, e = quasiParticle(h, Al, Ar, Ac, C, p, num)
print('First triplet: {}'.format(', '.join(str(v.real) for v in e)))